In [1]:
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

import _fix_paths
from lib.data import PEMSBay
from lib.models import STGCN_VAE

In [2]:
HIST_WINDOW = 12
PRED_WINDOW = 12
LATENT_DIM = 325
LEARNING_RATE = 0.01
WEIGHT_DECAY = 5e-4
NUM_EPOCHS = 200
BATCH_SIZE = 8
TEMPORAL_KERNEL = 3
SPATIAL_KERNEL = 3

In [3]:
train_set = PEMSBay('../datasets/PEMS-BAY', 'train', HIST_WINDOW, PRED_WINDOW)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/io/pytables.py:3007: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  index = factory(


In [4]:
model = STGCN_VAE(
    SPATIAL_KERNEL, TEMPORAL_KERNEL, [(HIST_WINDOW + PRED_WINDOW, 16, 64), (64, 32, 128)], LATENT_DIM
)

In [5]:
print(model)

STGCN_VAE(
  (encoder): ModuleList(
    (0): SpatioTemporalConv(
      (temporal_conv1): TemporalConv(
        (align): Align(
          (conv): Conv2d(24, 16, kernel_size=(1, 1), stride=(1, 1), padding=valid)
        )
        (gconv): ChebConv(24, 16, K=3, normalization=sym)
      )
      (spatial_conv): SpatialConv(
        (align): Align(
          (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), padding=valid)
        )
        (gconv): ChebConv(16, 16, K=3, normalization=sym)
      )
      (temporal_conv2): TemporalConv(
        (align): Align(
          (conv): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), padding=valid)
        )
        (gconv): ChebConv(16, 64, K=3, normalization=sym)
      )
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): SpatioTemporalConv(
      (temporal_conv1): TemporalConv(
        (align): Align(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model = model.to(device)

Using device: cpu


In [7]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

In [8]:
X, y = next(iter(train_loader))
X = X.to(device)
y = y.to(device)
edge_idx = train_set.edge_idx.to(device)
edge_wt = train_set.edge_wt.to(device)

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
loss_mse = nn.MSELoss()

In [10]:
X.shape, y.shape

(torch.Size([8, 1, 325, 12]), torch.Size([8, 1, 325, 12]))

In [11]:
model.train()
for epoch in tqdm(range(1, NUM_EPOCHS+1)):
    optimizer.zero_grad()
    y_hat = model(X, y, edge_idx, edge_wt, sample=True)
    loss = loss_mse(y, y_hat)
    print(f"[Epoch {epoch}/{NUM_EPOCHS}]: loss = {loss.item():.4f}")
    loss.backward()
    optimizer.step()

  0%|          | 0/200 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([8, 1, 325, 1])) that is different to the input size (torch.Size([8, 1, 325, 12])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[Epoch 1/201]: loss = 4519.2393
[Epoch 2/201]: loss = 4328.8296
[Epoch 3/201]: loss = 3966.7544
[Epoch 4/201]: loss = 2659.7573
[Epoch 5/201]: loss = 439.5177
[Epoch 6/201]: loss = 3649.3193
[Epoch 7/201]: loss = 846.6057
[Epoch 8/201]: loss = 1864.3613
[Epoch 9/201]: loss = 2198.8323
[Epoch 10/201]: loss = 2229.4685
[Epoch 11/201]: loss = 2057.2114
[Epoch 12/201]: loss = 1670.2412
[Epoch 13/201]: loss = 1129.2440
[Epoch 14/201]: loss = 674.4900
[Epoch 15/201]: loss = 347.0389
[Epoch 16/201]: loss = 402.9964
[Epoch 17/201]: loss = 398.0255
[Epoch 18/201]: loss = 298.7583
[Epoch 19/201]: loss = 375.0571
[Epoch 20/201]: loss = 305.1799
[Epoch 21/201]: loss = 315.4176
[Epoch 22/201]: loss = 295.1932
[Epoch 23/201]: loss = 265.1424
[Epoch 24/201]: loss = 268.0796
[Epoch 25/201]: loss = 270.4122
[Epoch 26/201]: loss = 253.3183
[Epoch 27/201]: loss = 282.4642
[Epoch 28/201]: loss = 247.5415
[Epoch 29/201]: loss = 252.8288
[Epoch 30/201]: loss = 268.6562
[Epoch 31/201]: loss = 232.6919
[Epoch

In [12]:
torch.save(model.state_dict(), 'trial_0_single_batch.pth')